In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Telomere_Length"
cohort = "GSE80435"

# Input paths
in_trait_dir = "../../input/GEO/Telomere_Length"
in_cohort_dir = "../../input/GEO/Telomere_Length/GSE80435"

# Output paths
out_data_file = "../../output/preprocess/Telomere_Length/GSE80435.csv"
out_gene_data_file = "../../output/preprocess/Telomere_Length/gene_data/GSE80435.csv"
out_clinical_data_file = "../../output/preprocess/Telomere_Length/clinical_data/GSE80435.csv"
json_path = "../../output/preprocess/Telomere_Length/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE80435-GPL10558_series_matrix.txt.gz', 'GSE80435-GPL6884_series_matrix.txt.gz', 'GSE80435_family.soft.gz']
Identified SOFT files: ['GSE80435_family.soft.gz']
Identified matrix files: ['GSE80435-GPL10558_series_matrix.txt.gz', 'GSE80435-GPL6884_series_matrix.txt.gz']

Background Information:
!Series_title	"Whole genome landscapes of major melanoma subtypes"
!Series_summary	"Cutaneous, acral and mucosal subtypes of melanoma were evaluated by whole-genome sequencing, revealing genes affected by novel recurrent mutations to the promoter (TERT, DPH3, OXNAD1, RPL13A, RALY, RPL18A, AP2A1), 5’-UTR (HNRNPUL1, CCDC77, PES1), and 3’-UTR (DYNAP, CHIT1, FUT9, CCDC141, CDH9, PTPRT) regions. TERT promoter mutations had the highest frequency of any mutation, but neither they nor ATRX mutations, associated with the alternative telomere lengthening mechanism, were correlated with greater telomere length. Genomic landscapes largely reflected ultraviolet radiation mutage

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, the only entry with index 0 contains
# 'region' values which don't have telomere length, age, or gender information
trait_row = None  # Telomere length data is not available
age_row = None    # Age data is not available
gender_row = None # Gender data is not available

# 2.2 Data Type Conversion Functions
# Although these won't be used since the data is not available, we define them as required
def convert_trait(value):
    """Convert telomere length values to float."""
    if not value or 'NA' in value or 'na' in value.lower():
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_age(value):
    """Convert age values to float."""
    if not value or 'NA' in value or 'na' in value.lower():
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if not value:
        return None
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# is_trait_available is False because trait_row is None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this substep because trait_row is None (clinical data is not available)


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651210', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651232', 'ILMN_1651237', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651279', 'ILMN_1651282', 'ILMN_1651285',
       'ILMN_1651288', 'ILMN_1651296', 'ILMN_1651315', 'ILMN_1651316'],
      dtype='object', name='ID')

Gene expression data shape: (28118, 27)


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols.
# These are specific probes used on Illumina microarray platforms and need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, 441782.0, 55818.0, nan, nan], 'GI': [23525203.0, 89042416.0, 46358420.0, 7376124.0, 5437312.0], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the gene expression data, we can see the identifiers are in the format "ILMN_XXXXXXX"
# From the gene annotation preview, we can see these identifiers are in the "ID" column
# The gene symbols are in the "Symbol" column

# 2. Extract the mapping between probe IDs and gene symbols
print("Creating gene mapping dataframe...")
prob_col = "ID"  # The column containing probe identifiers
gene_col = "Symbol"  # The column containing gene symbols

# Get the mapping dataframe using the function from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print some statistics about the mapping
print(f"Total number of probes in mapping: {len(gene_mapping)}")
print(f"Number of probes with gene symbols: {gene_mapping['Gene'].notna().sum()}")

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
print("Converting probe-level measurements to gene-level expression...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print shape and preview of the gene expression data after mapping
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


Creating gene mapping dataframe...
Total number of probes in mapping: 36750
Number of probes with gene symbols: 36750
Converting probe-level measurements to gene-level expression...
Gene expression data shape after mapping: (14757, 27)
First 10 gene symbols after mapping:
Index(['A1BG', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AACSL', 'AADACL1',
       'AADAT', 'AAMP'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# The gene_data already has proper gene symbols from step 6
# Now normalize these symbols using the NCBI Gene database information
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since we determined in Step 2 that trait data is not available (trait_row = None),
# we cannot properly link clinical and genetic data for this cohort

# Create an empty DataFrame with the correct sample IDs for proper evaluation
clinical_features = pd.DataFrame(index=gene_data_normalized.columns)
print(f"Clinical features shape: {clinical_features.shape}")

# Create a dummy dataframe with the necessary structure for validation
dummy_df = pd.DataFrame(index=gene_data_normalized.columns)
dummy_df['dummy_trait'] = 0  # Add a dummy column to satisfy the validation requirements

# 3-6. Since trait data is not available, we cannot create a properly linked dataset
# Validate and save this information using is_final=True
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=False,  # Setting this to False as determined in Step 2
    is_biased=False,  # Setting to False since we can't determine bias without trait data
    df=dummy_df,  # Using dummy dataframe with necessary structure
    note="Dataset contains gene expression data but lacks telomere length measurements necessary for the trait analysis."
)

print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")

# We don't save linked_data since the dataset is not usable for our analysis
print("No linked data saved due to missing trait information.")

Gene data shape before normalization: (14757, 27)
Gene data shape after normalization: (13877, 27)


Normalized gene data saved to ../../output/preprocess/Telomere_Length/gene_data/GSE80435.csv
Clinical features shape: (27, 0)
Abnormality detected in the cohort: GSE80435. Preprocessing failed.
Data quality check result: Not usable
No linked data saved due to missing trait information.
